In [1]:
import pandas as pd
import numpy as np
from keras.datasets import imdb
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from collections import Counter
from nltk.corpus import stopwords
import os
import re
import string
import matplotlib.pyplot as plt


/Users/ashish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ashish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ashish/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/ashish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/ashish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning

In [2]:
top_words = 5000
epoch_num = 5
batch_size = 64


In [3]:
df_train = pd.read_csv('data/train_kaggle.csv', sep=',', encoding='utf-8')

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
# df_train['text'].apply(type).unique()
# # explore more why the text type is coming as float

# print("Before removing float type: ",df_train.shape)
# # only remove rows with text as float type

# df_train = df_train.drop(df_train[df_train['text'].apply(type) == float].index)
# print("After removing float type: ", df_train.shape)

In [5]:
def textClean(text):
    """
    Get rid of the non-letter and non-number characters
    """
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return (text)

def cleanup(text):
    text = textClean(text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text

In [6]:
missing_rows = []
for i in range(len(df_train)):
    if df_train.loc[i, 'text'] != df_train.loc[i, 'text']:
        missing_rows.append(i)
df_train = df_train.drop(missing_rows).reset_index().drop(['index','id'],axis=1)


In [7]:
df_train.head(7)

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1


In [8]:
for i in range(len(df_train['text'])):
        df_train.loc[i, 'text'] = cleanup(df_train.loc[i,'text'])

In [9]:
#change above X if we r using tokenize and other nlp process
#Dropping the Nan values and info
#df_train.dropna(inplace=True)
X = df_train['text']
y = df_train['label']

# do the similar think on headline (author ?? or source)
# name these to test , validation
X_train,  X_test,  y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=12345)


In [10]:
cnt = Counter()
X_train_modified = []
for x in X_train:
    X_train_modified.append(x.split())
    for word in X_train_modified[-1]:
        cnt[word] += 1  

# Storing most common words
most_common = cnt.most_common(top_words + 1)
word_bank = {}
id_num = 1
for word, freq in most_common:
    word_bank[word] = id_num
    id_num += 1


In [11]:
# Encode the sentences
for news in X_train_modified:
    i = 0
    while i < len(news):
        if news[i] in word_bank:
            news[i] = word_bank[news[i]]
            i += 1
        else:
            del news[i]


In [12]:
y_train = list(y_train)
y_test = list(y_test)

# Delete the short news
i = 0
while i < len(X_train_modified):
    if len(X_train_modified[i]) > 10:
        i += 1
    else:
        del X_train_modified[i]
        del y_train[i]


In [13]:
# Generating test data
x_test = []
for x in X_test:
    x_test.append(x.split())

# Encode the sentences
for news in x_test:
    i = 0
    while i < len(news):
        if news[i] in word_bank:
            news[i] = word_bank[news[i]]
            i += 1
        else:
            del news[i]


In [19]:
# Truncate and pad input sequences
max_article_length = 500
X_train = sequence.pad_sequences(X_train_modified, maxlen=max_article_length)
X_test = sequence.pad_sequences(x_test, maxlen=max_article_length)

# Convert to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)


In [21]:
# Create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+2, embedding_vecor_length, input_length=max_article_length))


In [16]:
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch_num, batch_size=batch_size)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy= %.2f%%" % (scores[1]*100))

# Draw the confusion matrix
y_pred = model.predict_classes(X_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          160064    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 168,417
Trainable params: 168,417
Non-trainable params: 0
_________________________________________________________________
None
Train on 16030 samples, validate on 4153 samples
Epoch 1/5
16030/16030 [==============================] - 115s 7ms/step - loss: 0.3069 - acc: 0.8759 - val_loss: 0.1794 - val_acc: 0.9369
Epoch 2/5
16030/16030 [==============================] - 111s 7ms/step - loss: 0.1449 - acc: 0.9517 - val_loss: 0.1546 - val_acc: 0.9427
Epoch 3/5
16030/16030 [==============================] - 108s 7ms/step -

In [22]:
# Create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+2, embedding_vecor_length, input_length=max_article_length))



In [23]:
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epoch_num, batch_size=batch_size)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy= %.2f%%" % (scores[1]*100))

# Draw the confusion matrix
y_pred = model.predict_classes(X_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 64)           320128    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 353,217
Trainable params: 353,217
Non-trainable params: 0
_________________________________________________________________
None
Train on 16030 samples, validate on 4153 samples
Epoch 1/5
16030/16030 [==============================] - 124s 8ms/step - loss: 0.2945 - acc: 0.8843 - val_loss: 0.1732 - val_acc: 0.9412
Epoch 2/5
16030/16030 [==============================] - 121s 8ms/step - loss: 0.1701 - acc: 0.9410 - val_loss: 0.3655 - val_acc: 0.8274
Epoch 3/5
16030/16030 [==============================] - 118s 7ms/step -

Making a small change